In [42]:
import json
import pandas as pd

path = 'MoviesDataSet/'

# Load Data from the Kaggle dataset
movies = pd.read_csv(path + 'movies_metadata.csv')

# Drop junk columns
movies = movies.drop('belongs_to_collection', axis=1)
movies = movies.drop('homepage', axis=1)
movies = movies.drop('original_language', axis=1)
movies = movies.drop('original_title', axis=1)
movies = movies.drop('overview', axis=1)
movies = movies.drop('tagline', axis=1)
movies = movies.drop('poster_path', axis=1)
movies = movies.drop('production_companies', axis=1)
movies = movies.drop('production_countries', axis=1)
movies = movies.drop('video', axis=1)

# These columns might be helpful, but they're not reliable in this dataset
movies = movies.drop('revenue', axis=1)
movies = movies.drop('budget', axis=1)

# Filter out any unreleased movies
movies = movies[movies['status'] == 'Released']
movies = movies.drop('status', axis=1)

# Only evaluate movies with ratings similar to our Christmas movies.
# This is important since classic Christmas movies are popular and the minimum rating is a 5.4
movies = movies[movies['vote_average'] >= 5.4]
movies = movies[movies['vote_average'] <= 8.0]

# Filter out extremely popular movies. Max popularity for a Christmas movie is 17.3
movies['popularity'] = movies['popularity'].astype(float)
movies = movies[movies['popularity'] <= 18]

# Split the date into year / quarter / month components for easier analysis
movies['release_year'] = pd.DatetimeIndex(movies['release_date']).year
movies['release_quarter'] = pd.DatetimeIndex(movies['release_date']).quarter
movies['release_month'] = pd.DatetimeIndex(movies['release_date']).month

# Filter out movies from outside of the year range where our labelled data lives
movies = movies[movies['release_year'] >= 1940 ]
movies = movies[movies['release_year'] <= 2015 ]

# Filter out extremely long movies
movies['runtime'] = movies['runtime'].astype(int, errors='ignore')
movies = movies[movies['runtime'] <= 150]

# Convert ID to an int. Because why wouldn't it be?
movies['id'] = movies['id'].astype(int)

# Sort movies by title
movies = movies.sort_values('title')

movies.head()

c:\users\matte\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,genres,id,imdb_id,popularity,release_date,runtime,spoken_languages,title,vote_average,vote_count,release_year,release_quarter,release_month
23501,False,"[{'id': 99, 'name': 'Documentary'}]",267752,tt3060338,0.108531,2013-11-21,74.0,[],#chicagoGirl,7.0,1.0,2013.0,4.0,11.0
28042,False,"[{'id': 37, 'name': 'Western'}]",143747,tt0060697,0.634857,1966-12-18,104.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...","$1,000 on the Black",6.0,2.0,1966.0,4.0,12.0
15775,False,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",4204,tt1024733,2.326842,2008-01-01,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",$5 a Day,6.0,24.0,2008.0,1.0,1.0
36186,False,"[{'id': 18, 'name': 'Drama'}]",248268,tt2106284,0.984593,2014-01-10,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",$50K and a Call Girl: A Love Story,6.3,11.0,2014.0,1.0,1.0
14832,False,"[{'id': 16, 'name': 'Animation'}, {'id': 18, '...",19311,tt0790799,3.350555,2008-09-04,78.0,"[{'iso_639_1': 'en', 'name': 'English'}]",$9.99,6.0,28.0,2008.0,3.0,9.0


In [43]:
def extract_language(row):
    data = row['spoken_languages']
    row['Is English'] = 'English' in data
    return row

movies = movies.apply(extract_language, axis=1)

# Filter down to only English movies since all labelled movies are English
movies = movies[movies['Is English'] == True]

movies = movies.drop('spoken_languages', axis=1)
movies = movies.drop('Is English', axis=1)
movies.head()

,adult,genres,id,imdb_id,popularity,release_date,runtime,title,vote_average,vote_count,release_year,release_quarter,release_month
15775,False,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",4204,tt1024733,2.326842,2008-01-01,98.0,$5 a Day,6.0,24.0,2008.0,1.0,1.0
36186,False,"[{'id': 18, 'name': 'Drama'}]",248268,tt2106284,0.984593,2014-01-10,90.0,$50K and a Call Girl: A Love Story,6.3,11.0,2014.0,1.0,1.0
14832,False,"[{'id': 16, 'name': 'Animation'}, {'id': 18, '...",19311,tt0790799,3.350555,2008-09-04,78.0,$9.99,6.0,28.0,2008.0,3.0,9.0
22323,False,"[{'id': 99, 'name': 'Documentary'}]",95383,tt2258233,0.891024,2013-01-11,89.0,$ellebrity,5.5,9.0,2013.0,1.0,1.0
25192,False,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",252178,tt2614684,7.080550,2014-10-10,99.0,'71,6.7,414.0,2014.0,4.0,10.0


In [44]:
# Detect Genres
def extract_genres(row):
    data = row['genres']
    arr = json.loads(data.replace("'", "\""))
    for genre in arr:
        row['Is ' + genre['name']] = 1

    return row

movies = movies.apply(extract_genres, axis=1)

# Set our NA values for non-genre membership to 0
#movies = movies.fillna(0)

# Drop the no longer needed source column
movies = movies.drop('genres', axis=1)
movies.head()

,Is Action,Is Adventure,Is Animation,Is Comedy,Is Crime,Is Documentary,Is Drama,Is Family,Is Fantasy,Is Foreign,...,imdb_id,popularity,release_date,release_month,release_quarter,release_year,runtime,title,vote_average,vote_count
15775,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,...,tt1024733,2.326842,2008-01-01,1.0,1.0,2008.0,98.0,$5 a Day,6.0,24.0
36186,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,tt2106284,0.984593,2014-01-10,1.0,1.0,2014.0,90.0,$50K and a Call Girl: A Love Story,6.3,11.0
14832,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,tt0790799,3.350555,2008-09-04,9.0,3.0,2008.0,78.0,$9.99,6.0,28.0
22323,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,tt2258233,0.891024,2013-01-11,1.0,1.0,2013.0,89.0,$ellebrity,5.5,9.0
25192,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,tt2614684,7.080550,2014-10-10,10.0,4.0,2014.0,99.0,'71,6.7,414.0


In [45]:
# Convert our adult column from true / false to 1 / 0.
# Note we move from String -> Bool -> Int here
movies['adult'] = movies['adult'].astype(bool).astype(int)

In [46]:
# These are IMDB IDs of movies known to be Christmas movies or classic Christmas movies. See other experiments for details.
labelled_movies = ['tt0993789','tt6433880','tt2402927','tt0081793','tt2709692','tt0064349','tt0041473','tt7736496','tt0032981','tt0043733','tt0338348','tt0117372','tt1430607','tt0099487','tt1401143','tt0086465','tt3530002','tt0044008','tt1067106','tt0037595','tt0096061','tt0116705','tt0457939','tt0034862','tt0087363','tt0039628','tt0110527','tt0033045','tt2083355','tt0039190','tt0319343','tt0085936','tt0047673','tt0104940','tt0095016','tt0058536','tt0097958','tt0107688','tt0114924','tt0111070','tt0085334','tt1268799','tt0037059','tt0314331','tt0071222','tt0060345','tt0099785','tt0373469','tt3850590','tt0059026','tt0038650','tt3824458','tt0307987']

# We need to label all rows based on if they're a Christmas movie or not so we can train a model
def set_label(row):
    imdb = row['imdb_id']
    if imdb in labelled_movies:
        row['Is Christmas Movie'] = 1
    else:
        row['Is Christmas Movie'] = 0
    return row

movies = movies.apply(set_label, axis=1)

movies.head()

,Is Action,Is Adventure,Is Animation,Is Comedy,Is Crime,Is Documentary,Is Drama,Is Family,Is Fantasy,Is Foreign,...,popularity,release_date,release_month,release_quarter,release_year,runtime,title,vote_average,vote_count,Is Christmas Movie
15775,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,...,2.326842,2008-01-01,1.0,1.0,2008.0,98.0,$5 a Day,6.0,24.0,0
36186,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,0.984593,2014-01-10,1.0,1.0,2014.0,90.0,$50K and a Call Girl: A Love Story,6.3,11.0,0
14832,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,3.350555,2008-09-04,9.0,3.0,2008.0,78.0,$9.99,6.0,28.0,0
22323,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0.891024,2013-01-11,1.0,1.0,2013.0,89.0,$ellebrity,5.5,9.0,0
25192,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,7.080550,2014-10-10,10.0,4.0,2014.0,99.0,'71,6.7,414.0,0


In [47]:
# Load our Keywords dataset
keywords = pd.read_csv(path + "keywords.csv")
keywords['id'] = keywords['id'].astype(int)

id           int32
keywords    object
dtype: object

In [48]:
# Merge the keywords column into our dataset
movies = pd.merge(movies, keywords, on='id')
movies.head()

,Is Action,Is Adventure,Is Animation,Is Comedy,Is Crime,Is Documentary,Is Drama,Is Family,Is Fantasy,Is Foreign,...,release_date,release_month,release_quarter,release_year,runtime,title,vote_average,vote_count,Is Christmas Movie,keywords
0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,...,2008-01-01,1.0,1.0,2008.0,98.0,$5 a Day,6.0,24.0,0,[]
1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,2014-01-10,1.0,1.0,2014.0,90.0,$50K and a Call Girl: A Love Story,6.3,11.0,0,[]
2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,2014-01-10,1.0,1.0,2014.0,90.0,$50K and a Call Girl: A Love Story,6.3,11.0,0,[]
3,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,2008-09-04,9.0,3.0,2008.0,78.0,$9.99,6.0,28.0,0,"[{'id': 10183, 'name': 'independent film'}, {'..."
4,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,2013-01-11,1.0,1.0,2013.0,89.0,$ellebrity,5.5,9.0,0,"[{'id': 208403, 'name': 'celebrity photographe..."


In [49]:
# Filter rows without keywords
movies = movies[movies['keywords'] != '[]']

In [50]:
# Detect Keywords
def extract_keywords(row):
    data = row['keywords']

    # Sorry, I suck at RegEx right now. The source data doesn't truly use JSON and doesn't encode apostrophes either
    data = data.replace("n's","ns") # Strip apostrophes not for serialization
    data = data.replace("r's","rs") # Strip apostrophes not for serialization
    data = data.replace("ladies'","ladies") # Strip apostrophes not for serialization
    data = data.replace("one's","ones") # Strip apostrophes not for serialization
    data = data.replace("d's","ds") # Strip apostrophes not for serialization
    data = data.replace("t's","ts") # Strip apostrophes not for serialization
    data = data.replace("l's","ls") # Strip apostrophes not for serialization
    data = data.replace("e's","es") # Strip apostrophes not for serialization
    data = data.replace("d'e","de") # Strip apostrophes not for serialization
    data = data.replace("girls' ","girls ") # Strip apostrophes not for serialization
    data = data.replace("boys' ","boys ") # Strip apostrophes not for serialization
    data = data.replace("y's","ys") # Strip apostrophes not for serialization
    data = data.replace("'s ","s") # Strip apostrophes not for serialization
    data = data.replace("ers' ","ers ") # Strip apostrophes not for serialization
    data = data.replace("s' ","s ") # Strip apostrophes not for serialization
    data = data.replace("'n'","n") # Strip apostrophes not for serialization
    data = data.replace("\\xa0","") # Weird data
    data = data.replace("p's", "ps")
    data = data.replace("o'h", "oh")
    data = data.replace("a'u", "au")
    data = data.replace("u'v", "uv")
    data = data.replace("d'a", "da")

    # The source data is only quasi-JSON so I need to change ' to " so it will parse
    data = data.replace("'", "\"")

    # These tags have to do with the labelled (Christmas movies) data
    # The full set of tags results in 14,400 columns and is too large to handle in this experiment
    # Note that a few of the things in this list are surprising and are likely due to the inclusion of Bad Santa and things like that
    labelled_tags = ['Angel','Bars and Restaurants','Burglar','Based on Novel','Chicago','Christmas','Christmas Party','Christmas Eve','Christmas Carol','Department Store','Female Nudity','Ghost','Gift','Holiday','Hoodlum','Hospital','Lawyer','LGBT','London England','Los Angeles','Love at First Sight','Monster','Multiple Storylines','Murder','Musical','New Years Eve','North Pole','Puppet','Road Trip','Santa Claus','Scrooge','Sex','Snow','Victorian England','Winter','Woman Director']
    
    # Because people can define Christmas movies by the things they're NOT, add in other common tags
    # this allows the algorithm to penalize movies with these common tags
    unlablled_tags = ['aftercreditsstinger','alien','based on a true story','biography','blood','corruption','dark comedy','death','detective','dog','drug','dystopia','england','escape','family','father','film noir','friendship','gangster','gay','high school','independent film','investigation','kidnapping','love','marriage','martial arts','money','music','new york','nudity','paris','party','prison','revenge','robbery','romance','secret','remake','sequel','serial killer','small town','sport','spy','stand-up comedy','suicide','suspense','teacher','teenager','vampire','violence','wedding','world war ii']

    try:
        arr = json.loads(data)
        for item in arr:
            if item in labelled_tags or item in unlablled_tags:
                row['Tag: ' + item['name']] = 1
    except:
        print('Could not parse: ' + data)

    return row

movies = movies.apply(extract_keywords, axis=1)

# Set our NA values for non-keyword membership to 0
#movies = movies.fillna(0)

# Drop the no longer needed source column
movies = movies.drop('keywords', axis=1)
movies.head()

,Is Action,Is Adventure,Is Animation,Is Christmas Movie,Is Comedy,Is Crime,Is Documentary,Is Drama,Is Family,Is Fantasy,...,imdb_id,popularity,release_date,release_month,release_quarter,release_year,runtime,title,vote_average,vote_count
3,NaN,NaN,1.0,0,NaN,NaN,NaN,1.0,NaN,NaN,...,tt0790799,3.350555,2008-09-04,9.0,3.0,2008.0,78.0,$9.99,6.0,28.0
4,NaN,NaN,NaN,0,NaN,NaN,1.0,NaN,NaN,NaN,...,tt2258233,0.891024,2013-01-11,1.0,1.0,2013.0,89.0,$ellebrity,5.5,9.0
5,1.0,NaN,NaN,0,NaN,NaN,NaN,1.0,NaN,NaN,...,tt2614684,7.080550,2014-10-10,10.0,4.0,2014.0,99.0,'71,6.7,414.0
6,1.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,tt0074080,0.908897,1974-06-01,6.0,2.0,1974.0,88.0,'Gator Bait,6.5,2.0
8,NaN,NaN,1.0,0,NaN,NaN,NaN,NaN,1.0,NaN,...,tt0208654,1.353059,1974-12-08,12.0,4.0,1974.0,25.0,'Twas the Night Before Christmas,5.9,11.0


In [51]:
# These pieces of logic were used to analyzed present tags
#stats = movies.describe(include='all')
#stats.to_csv('Stats.csv')

#stats = movies[movies['Is Christmas Movie'] == 1].describe(include='all')
#stats.to_csv('LabelledStats.csv')

In [36]:
# Set the index for clarity
movies_export = movies.set_index('id')

# Export the full data set
movies_export.to_csv('Processed.csv')

In [37]:
# These columns are helpful, but bias the ML algorithms too much given the quantity of tiny movies
movies_export = movies_export.drop('popularity', axis=1)
movies_export = movies_export.drop('vote_count', axis=1)
movies_export = movies_export.drop('vote_average', axis=1)

movies_export.head()

,Is Action,Is Adventure,Is Animation,Is Christmas Movie,Is Comedy,Is Crime,Is Documentary,Is Drama,Is Family,Is Fantasy,...,Tag: 绝地奶霸,Tag: 超级妈妈,adult,imdb_id,release_date,release_month,release_quarter,release_year,runtime,title
id,,,,,,,,,,,,,,,,,,,,,
4204,0.0,0.0,0.0,0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1,tt1024733,2008-01-01,1.0,1.0,2008.0,98.0,$5 a Day
248268,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1,tt2106284,2014-01-10,1.0,1.0,2014.0,90.0,$50K and a Call Girl: A Love Story
248268,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1,tt2106284,2014-01-10,1.0,1.0,2014.0,90.0,$50K and a Call Girl: A Love Story
19311,0.0,0.0,1.0,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1,tt0790799,2008-09-04,9.0,3.0,2008.0,78.0,$9.99
95383,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1,tt2258233,2013-01-11,1.0,1.0,2013.0,89.0,$ellebrity


In [38]:
id_columns = ['title', 'imdb_id']

labelled = movies_export[movies_export['Is Christmas Movie'] == 1]
labelled.to_csv('Labelled.csv')

dieHard = movies_export[movies_export['title'] == 'Die Hard']
dieHard = dieHard.drop(id_columns, axis=1) # Remove the title
dieHard.to_csv('DieHard.csv')

train_test = movies_export[movies_export['title'] != 'Die Hard']
train_test = train_test.drop(id_columns, axis=1) # Remove the title
train_test.to_csv('traintest.csv')